<a href="https://colab.research.google.com/github/abakamousa/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -qqq openai
#!pip install openai==0.28
!pip install -Uqqq chromadb
!pip install langchain-community langchain-core
!pip install datasets
!pip install tiktoken
#!pip install --upgrade openai





In [2]:
import os
import textwrap

import langchain
import chromadb
import transformers
import openai
import torch

from transformers import AutoTokenizer
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [4]:
os.environ["OPENAI_API_KEY"] = "XXXXX"

In [5]:
#Set up HuggingFace Pipeline with Llama-2-7b-chat-hf model
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
      "text-generation", #task
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
)

#LLM intialized in HuggingFace Pipeline wrapper
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

<ipython-input-5-a2d426da90c7>:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [11]:
# Load documents locally as CSV
loader = CSVLoader('data.csv')
docs = loader.load()
docs[0]

Document(metadata={'source': 'data.csv', 'row': 0}, page_content='\ufeffQuestion;Answer: I can no longer afford purchase {{Order Number}}\nNone: cancel it;"\'I\\\'m fully aware that you\\\'re seeking assistance with canceling your purchase with the purchase number {{Order Number}}. We highly value your satisfaction and I\\\'m here to guide you through the cancellation process. To ensure a smooth cancellation experience,please follow these steps:\\n\\n1. Log in to your account: Start by accessing our platform and logging in to your {{Company Account}}.\\n2. Navigate to your order history: Once you\\\'re logged in,go to the ""{{Online Order Interaction}}"" or ""{{Online Order Interaction}}"" section.\\n3. Locate your purchase: Look for the purchase associated with the purchase number {{Order Number}} and click on it for detailed information.\\n4. Initiate the cancellation: You should find the option to ""{{Cancel Purchase}}"" associated with your purchase,please select it.\\n5. Follow an

In [12]:
# Split document into text chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

In [13]:
# Initialize the open-source embedding function, default: text-embedding-ada-002
embedding_function = OpenAIEmbeddings()

<ipython-input-13-cd7ba391d3df>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding_function = OpenAIEmbeddings()


In [17]:
# Load it into ChromaDB
db = Chroma.from_documents(docs, embedding_function)

In [19]:
#Design Prompt Template
template = """I need assistance cancelling order {{Order Number}}.

{context}

Answer the customer's questions only using the source data provided. If you are unsure, say "I don't know, please call our customer support". Use engaging, courteous, and professional language similar to a customer representative.
Keep your answers concise.

Question:

Answer: """

In [20]:
#Intiliaze prompt using prompt template via LangChain
prompt = PromptTemplate(template=template, input_variables=["context"])
print(
    prompt.format(
        context = "A customer is on the ecommerce website and wants to chat with the website chatbot."
    )
)

I need assistance cancelling order {Order Number}.

A customer is on the ecommerce website and wants to chat with the website chatbot.

Answer the customer's questions only using the source data provided. If you are unsure, say "I don't know, please call our customer support". Use engaging, courteous, and professional language similar to a customer representative.
Keep your answers concise.

Question:

Answer: 


In [21]:
#Chain to have all components together and query the LLM
chain_type_kwargs = {"prompt": prompt}

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
# Formatted printing
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=80)))

#Running chain through LLM with query
query = "Can I get a refund ?"
response = chain.run(query)
print_response(response)

<ipython-input-22-44903dc171b5>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run(query)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
